## Data Ingestion for Deep RAG

In this notebook, we'll load extracted data into Qdrant vector database:

- **Markdown**: Page-level chunks with metadata
- **Tables**: Separate documents with context and page numbers
- **Images**: Text descriptions embedded (generated in notebook 06-01b)
- **Hybrid Search**: Dense (semantic) + Sparse (keyword) embeddings

**Prerequisites:**
- Run notebook 06-01 first to extract PDFs
- Run notebook 06-01b to generate image descriptions
- Qdrant server running on localhost:6333
- Google API key set in .env file

**Output:**
- Single Qdrant collection with all content types
- Rich metadata for filtering (company, year, quarter, doc_type, page)
- Deduplication using file hashes

### 1. Setup and Imports

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import hashlib
from pathlib import Path

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_qdrant import QdrantVectorStore, RetrievalMode, FastEmbedSparse
from langchain_core.documents import Document
from qdrant_client import QdrantClient

### 2. Configuration

In [ ]:
# Paths
MARKDOWN_DIR = "data/rag-data/markdown"
TABLES_DIR = "data/rag-data/tables"
IMAGES_DESC_DIR = "data/rag-data/images_desc"

# Qdrant Configuration
COLLECTION_NAME = "financial_docs"
EMBEDDING_MODEL = "models/gemini-embedding-001"

### 3. Initialize Embeddings and Client

In [ ]:
# Embeddings
embeddings = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL)
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

### 4. Create or Recreate Collection

In [ ]:
# Create vector store
vector_store = QdrantVectorStore.from_documents(
    documents=[],  # Empty initialization
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    url="http://localhost:6333",
    collection_name=COLLECTION_NAME,
    retrieval_mode=RetrievalMode.HYBRID,
    force_recreate=True
)

In [ ]:
vector_store._client

### 5. Helper Functions

In [ ]:
# Reuse from 06-01
def extract_metadata_from_filename(filename: str):
    """Extract metadata from filename."""
    
    filename = filename.replace('.pdf', '').replace('.md', '')
    parts = filename.split()

    return {
        'company_name': parts[0],
        'doc_type': parts[1],
        'fiscal_quarter': parts[2] if len(parts)==4 else None,
        'fiscal_year': parts[-1]
    }

In [ ]:
def compute_file_hash(file_path: Path) -> str:
    """Compute SHA-256 hash for deduplication."""
    sha256_hash = hashlib.sha256()
    with open(file_path, 'rb') as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

In [ ]:
all_points = vector_store.client.scroll(
        collection_name=COLLECTION_NAME,
        limit=10000,
        with_payload=True
    )
    
all_points[0][0].payload['metadata']['file_hash']

In [ ]:
def get_processed_hashes():
    """Get file hashes already in Qdrant."""
    
    all_points = vector_store.client.scroll(
        collection_name=COLLECTION_NAME,
        limit=10000,
        with_payload=True
    )
    
    return set(
        point.payload['metadata'].get('file_hash') 
        for point in all_points[0]
    )

### 6. Ingestion Function

In [ ]:
def ingest_file(file_path: Path, processed_hashes: set):
    """Ingest any markdown file - automatically detects type and handles accordingly."""
    
    file_hash = compute_file_hash(file_path)
    if file_hash in processed_hashes:
        return
    
    # Determine content type from path
    path_parts = file_path.parts
    if 'markdown' in path_parts:
        content_type = 'text'
        doc_name = file_path.name
    elif 'tables' in path_parts:
        content_type = 'table'
        doc_name = file_path.parent.name + '.md'
    elif 'images_desc' in path_parts:
        content_type = 'image'
        doc_name = file_path.parent.name + '.md'
    else:
        return  # Unknown type, skip
    
    # Read content and prepare metadata
    content = file_path.read_text(encoding='utf-8')
    file_metadata = extract_metadata_from_filename(doc_name)
    
    # Build base metadata (common for all documents)
    base_metadata = file_metadata.copy()
    base_metadata['content_type'] = content_type
    base_metadata['file_hash'] = file_hash
    base_metadata['source_file'] = file_path.name
    
    # Process content into (text, page_num) chunks
    chunks = []
    if content_type == 'text':
        # Split markdown by page breaks
        pages = content.split("<!-- page break -->")
        chunks = [(text.strip(), i) for i, text in enumerate(pages, start=1) if text.strip()]
    else:
        # Extract page number from filename (table_1_page_5.md or page_5.md)
        page_num = None
        if '_page_' in file_path.stem:
            page_num = int(file_path.stem.split('_page_')[1])
        chunks = [(content, page_num)]
    
    # Create documents with unified logic
    documents = []
    for text, page_num in chunks:
        metadata = base_metadata.copy()
        if page_num:
            metadata['page'] = page_num
        documents.append(Document(page_content=text, metadata=metadata))
    
    # Add to vector store
    if documents:
        vector_store.add_documents(documents)
        processed_hashes.add(file_hash)

In [ ]:
### 7. Ingest All Data

In [ ]:
processed_hashes = get_processed_hashes()

# Get all markdown files from all directories
base_path = Path("data/rag-data")
all_md_files = list(base_path.rglob("*.md"))

# Ingest everything with single function
for md_file in all_md_files:
    ingest_file(md_file, processed_hashes)

print("All data ingested successfully")

### 8. Verify Ingestion

In [ ]:
collection_info = vector_store.client.get_collection(COLLECTION_NAME)
collection_info

### 9. Test Search

In [ ]:
# Test hybrid search
query = "What is Amazon's revenue?"
results = vector_store.similarity_search(query, k=5)

results